In [1]:
#| default_exp main
import sys,os
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [2]:
#| export
import json
from datetime import datetime
from pathlib import Path
from typing import Union

import pandas as pd
import pyodbc
from dotenv import find_dotenv, load_dotenv
from fastcore.test import *
from pymongo import MongoClient
from rich import print

from extracao.format import _filter_matlab, _format_matlab, merge_on_frequency

load_dotenv(find_dotenv(), override=True)

True

In [3]:
#|eval: false
folder = Path.cwd().parent / 'extracao' / 'datasources' / 'arquivos' / 'saida'
folder.ls()

(#22) [Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/aero_discarded.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/AnatelDB.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/arquitetura.md'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/base.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/canalização_smp.xlsx'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/dados.md'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/estacoes.parquet.gzip'),Path('c:/Users/rsilva/OneDrive - ANATEL/Code/anateldb/extracao/datasources/arquivos/saida/estacoes_disca

In [25]:
#|eval: false
#|code_fold: true

def read_df(name: str, dtype_backend='pyarrow')->pd.DataFrame:
    return pd.read_parquet(folder / f'{name}.parquet.gzip', dtype_backend=dtype_backend)# .astype("category")

In [34]:
estacoes = read_df('srd', 'numpy_nullable')
estacoes

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,248,[],-1.0,-1.0,1200203,Cruzeiro do Sul,AC,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.5,-1,-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
1,539.0,X-MEDIAGROUP S.A.,50410887137,248,-1,-1.0,-1.0,1200336,Mâncio Lima,AC,...,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.5,[],-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
2,79.0,TELEVISAO OESTE BAIANO LTDA,6030116240,248,322647029,-12.101389,-44.99361,2903201,Barreiras,BA,...,1000.0,3.2,0.0,0.0,44.700001,0.77,0.5,"{'0': 0, '10': 0.34874388613190943, '20': 0.94...",50.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
3,69.0,TELEVISAO SANTA CRUZ LTDA,6020355110,248,322623553,-14.779445,-39.262222,2914802,Itabuna,BA,...,5000.0,2.79,0.0,55.0,86.0,0.62,0.25,"{'0': 0.22, '10': 0.0, '20': 0.22, '30': 0.77,...",88.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
4,177.0,TV CABRALIA LTDA,6020354903,248,322623537,-14.783334,-39.283333,2914802,Itabuna,BA,...,4700.0,3.22,0.0,330.0,67.410004,1.18,0.5,"{'0': 1.01, '10': 1.51, '20': 1.51, '30': 1.01...",75.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31158,587.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446946575,801,[],-1.0,-1.0,5107859,São Félix do Araguaia,MT,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
31159,497.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446946737,801,[],-1.0,-1.0,1200500,Sena Madureira,AC,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
31160,641.0,RF TECNOLOGIA E PARTICIPACOES LTDA,50446951900,801,[],-1.0,-1.0,5210109,Ipameri,GO,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...
31161,527.0,TV CIDADE PRODUCOES LTDA,50446952036,801,[],-1.0,-1.0,2102036,Bom Jesus das Selvas,MA,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,http://sistemas.anatel.gov.br/se/eApp/reports/...


In [35]:
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31163 entries, 0 to 31162
Data columns (total 29 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Frequência                   31163 non-null  Float64 
 1   Entidade                     31163 non-null  string  
 2   Fistel                       31163 non-null  Int64   
 3   Serviço                      31163 non-null  Int16   
 4   Estação                      31163 non-null  string  
 5   Latitude                     31163 non-null  Float32 
 6   Longitude                    31163 non-null  Float32 
 7   Código_Município             31163 non-null  Int32   
 8   Município                    31163 non-null  category
 9   UF                           31163 non-null  category
 10  Classe                       31163 non-null  category
 11  Classe_Emissão               31163 non-null  category
 12  Largura_Emissão(kHz)         31163 non-null  Float32 
 13  V

In [28]:
estacoes = estacoes.astype('string').replace('-1.0', '-1').astype('category')
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Frequência                   979522 non-null  category
 1   Entidade                     979522 non-null  category
 2   Fistel                       979522 non-null  category
 3   Serviço                      979522 non-null  category
 4   Estação                      979522 non-null  category
 5   Latitude                     979522 non-null  category
 6   Longitude                    979522 non-null  category
 7   Código_Município             979522 non-null  category
 8   Município                    979522 non-null  category
 9   UF                           979522 non-null  category
 10  Classe                       979522 non-null  category
 11  Classe_Emissão               979522 non-null  category
 12  Largura_Emissão(kHz)         979522 non-null

In [29]:
estacoes.sample(10)

,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
495426,459.8125,ORION ALARMES MONITORADOS LTDA.,50411424408,19,1004053433,-25.237005,-50.59347,4110102,Imbituva,PR,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
326503,168.41875,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943,19,1002189869,-19.760183,-43.880413,3157807,Santa Luzia,MG,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
297653,168.23125,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943,19,1001758606,-19.857714,-43.92029,3106200,Belo Horizonte,MG,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
247253,167.90625,BAHIA SECRETARIA DA SEGURANÇA PUBLICA,6030089684,19,697664546,-12.249047,-38.96013,2910800,Feira de Santana,BA,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
396290,258.125,TELEMAR NORTE LESTE S.A. EM RECUPERACAO JUDICIAL,4030120318,175,1003867518,-20.295528,-42.805,3157401,Santa Cruz do Escalvado,MG,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
762518,1940.0,CLARO S.A.,50409889580,10,690475063,-18.7646,-51.3485,5211305,Itarumã,GO,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
205826,167.64375,SECRETARIA DE ESTADO DA SEGURANÇA PUBLICA,50401415481,19,1002151500,-2.531544,-44.10757,2107506,Paço do Lumiar,MA,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
835491,2655.0,TIM S A,50409428698,10,695211323,-11.3797,-40.0089,2906873,Capim Grosso,BA,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
745132,1875.0,CLARO S.A.,50409105090,10,1015141967,-24.2764,-47.2354,3537206,Pedro de Toledo,SP,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
227461,167.78125,POLICIA MILITAR DO ESTADO DE MINAS GERAIS,50401288943,19,1008041669,-19.60436,-44.02789,3149309,Pedro Leopoldo,MG,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [32]:
estacoes.to_parquet(folder / 'estacoes.parquet.gzip', compression='gzip', index=False)

In [33]:
estacoes.Fonte.value_counts()

Fonte
MOSAICO-LICENCIAMENTO                              817750
CANALIZACAO-SMP                                    119408
STEL                                                19599
MOSAICO-SRD                                         14855
SRD                                                  4965
STEL | ICAO                                           904
ICAO                                                  690
CANALIZACAO-VOR/ILS/DME                               264
MOSAICO-LICENCIAMENTO | ICAO | AISWEB                 173
AISGEO                                                169
RADAR                                                 152
MOSAICO-LICENCIAMENTO | ICAO                          152
STEL | ICAO | AISWEB                                  142
AISWEB                                                 58
MOSAICO-LICENCIAMENTO | AISGEO                         52
STEL | AISWEB                                          35
ICAO | AISWEB                                          32
ICAO | A

In [7]:
estacoes['Frequência'] = estacoes.Frequência.astype('category')
estacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype                                                       
---  ------                       --------------   -----                                                       
 0   Frequência                   979522 non-null  category                                                    
 1   Entidade                     979522 non-null  string[pyarrow]                                             
 2   Fistel                       979522 non-null  uint64[pyarrow]                                             
 3   Serviço                      979522 non-null  uint16[pyarrow]                                             
 4   Estação                      979522 non-null  string[pyarrow]                                             
 5   Latitude                     979522 non-null  float[pyarrow]                                        

In [12]:
cat = estacoes.astype('category').copy()
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 979522 entries, 0 to 979521
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   Frequência                   979522 non-null  category
 1   Entidade                     979522 non-null  category
 2   Fistel                       979522 non-null  category
 3   Serviço                      979522 non-null  category
 4   Estação                      979522 non-null  category
 5   Latitude                     979522 non-null  category
 6   Longitude                    979522 non-null  category
 7   Código_Município             979522 non-null  category
 8   Município                    979522 non-null  category
 9   UF                           979522 non-null  category
 10  Classe                       979522 non-null  category
 11  Classe_Emissão               979522 non-null  category
 12  Largura_Emissão(kHz)         979522 non-null

In [13]:
cat.to_parquet(folder / 'estacoes_cat.parquet.gzip', index=False, compression='gzip')

In [42]:
estacoes.Fonte.value_counts()

Fonte
MOSAICO-LICENCIAMENTO                              817750
CANALIZACAO-SMP                                    119408
STEL                                                19599
MOSAICO-SRD                                         14855
SRD                                                  4965
STEL | ICAO                                           904
ICAO                                                  690
CANALIZACAO-VOR/ILS/DME                               264
MOSAICO-LICENCIAMENTO | ICAO | AISWEB                 173
AISGEO                                                169
MOSAICO-LICENCIAMENTO | ICAO                          152
RADAR                                                 152
STEL | ICAO | AISWEB                                  142
AISWEB                                                 58
MOSAICO-LICENCIAMENTO | AISGEO                         52
STEL | AISWEB                                          35
ICAO | AISWEB                                          32
ICAO | A

In [43]:
for c in [
	'ICAO',
	'CANALIZACAO-VOR/ILS/DME',
	'AISGEO',
	'RADAR',
	'ICAO | AISWEB',
	'AISWEB',
	'ICAO | AISGEO',
	'REDEMET',
]:
	for col in [
		'Fistel',
		'Serviço',
		'Estação',
		'Classe',
		'Classe_Emissão',
		'Largura_Emissão(kHz)',
		'Validade_RF',
		'Status',
		'Fonte',
		'Multiplicidade',
		'Cota_Base_Torre(m)',
		'Potência_Transmissor(W)',
		'Ganho_Antena(dBd)',
		'Ângulo_Elevação_Antena',
		'Azimute_Antena',
		'Altura_Antena(m)',
		'Atenuação_Linha(db/100m)',
		'Perdas_Acessórias_Linha(db)',
		'Padrão_Antena(dBd)',
		'Comprimento_Linha(m)',
		'Relatório_Canal',
	]:
		assert estacoes.loc[estacoes.Fonte == c, col].nunique() == 1


In [44]:
for c in [
	'STEL | ICAO',
	'MOSAICO-LICENCIAMENTO | ICAO | AISWEB',
	'MOSAICO-LICENCIAMENTO | ICAO',
	'STEL | ICAO | AISWEB',
	'STEL | AISWEB',
	'MOSAICO-LICENCIAMENTO | AISWEB',
]:
	print(estacoes.loc[estacoes.Fonte == c, ['Fistel', 'Serviço', 'Estação']].sample(5))


Fistel  Serviço     Estação
37302  50415208017      507  1011658779
37214  50405707169      507  1009310221
36845  50004641507      507  1006018902
36467  50415723132      507  1007719025
38021  50415723132      507  1007724282

Fistel  Serviço    Estação
35502  11030016470       19  699526159
35556  11030016470       19  699526302
35867  11030016470       19  699525314
35614  50013784196       19  691104018
35541  11020237015       19    1068946

Fistel  Serviço     Estação
37611  50413129829       19  1002318553
37022   2030398152       19   687305675
35644  50406881138       19   692616217
37077  50404959350       19   690327978
37106  50411781430       19  1000665175

Fistel  Serviço     Estação
35878  50415723132      507  1007685023
36107  50415723132      507  1007707558
37902  50415723132      507  1007541412
35602  50415723132      507  1007688634
37930  50415723132      507  1007661485

Fistel  Serviço     Estação
35933  50415723132      507  1007558129
36102  50415723132      507  1007640518
35621  50415723132      507  1007541412
36122  50415723132      507  1007688634
36851  50415699916      507  1005348232

Fistel  Serviço     Estação
36427  11030016470       19   699525039
27001  50444239804      108  1000823587
35694  11030016470       19   699525250
27009  50444239804      108  1000823501
27008  50444239804      108  1000823439

In [ ]:
df

,Frequência,Entidade_x,Fistel,Serviço,Estação,Latitude_x,Longitude_x,Código_Município,Município,UF,...,Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal,Latitude_y,Longitude_y,Entidade_y,Fonte_y,Multiplicidade_y,_merge
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066219,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1,right_only
1066220,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1,right_only
1066221,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1,right_only
1066222,995.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1,right_only


In [ ]:
left_suffix, right_suffix = '_x', '_y'
lat, long = 'Latitude', 'Longitude'

left_only = df._merge == 'left_only'
right_only = df._merge == 'right_only'
both = df._merge == 'both'
df = df.drop(columns=['_merge'])

In [ ]:
only_left = df[left_only].copy()
only_left = only_left.iloc[:, :len(df_left.columns)]
only_left.columns = df_left.columns
only_left


,Frequência,Entidade,Fistel,Serviço,Estação,Latitude,Longitude,Código_Município,Município,UF,...,Potência_Transmissor(W),Ganho_Antena(dBd),Ângulo_Elevação_Antena,Azimute_Antena,Altura_Antena(m),Atenuação_Linha(db/100m),Perdas_Acessórias_Linha(db),Padrão_Antena(dBd),Comprimento_Linha(m),Relatório_Canal
0,87.5,ASSOCIACAO COMUNITARIA VOZ DA LIBERDADE DE TUR...,50415095220,231,1008210959,-2.228611111111,-45.306666666666665,2112456,Turilândia,MA,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,87.5,ASSOCIACAO COMUNITARIA DO MORAD.DE ALVORADA DE...,50409064718,231,699491851,-18.734166666666667,-43.36472222222217,3102407,Alvorada de Minas,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,87.5,ASSOCIAÇÃO COMUNITARIA DA JUVENTUDE DE CONGONH...,50405625782,231,699359830,-18.812777777777665,-43.673611111111,3118106,Congonhas do Norte,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,87.5,ASSOCIACAO COMUNITARIA FOLHETA,50404381251,231,690859562,-18.95,-43.266666666666666,3122603,Dom Joaquim,MG,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,87.5,ASSOCIAÇÃO DE RÁDIO COMUNITÁRIA DE CASTANHEIRA...,50411566547,231,1008401606,-11.137222222222167,-58.61333333333334,5102850,Castanheira,MT,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065599,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578503,-6.31095,-38.2823,2406007,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065600,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578511,-6.40588,-38.3899,2407005,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065601,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578520,-6.45823,-38.182,2414100,<NA>,RN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1065602,1765.0,TELEFONICA BRASIL S.A.,50409146285,010,1011578538,-5.71287,-38.1578,2311231,<NA>,CE,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
only_right = df[right_only].copy()
cols = ['Frequência'] + df.columns[len(df_left.columns):].to_list()
only_right = only_right.loc[:, cols]
only_right.columns = df_right.columns
only_right

,Frequência,Latitude,Longitude,Entidade,Fonte,Multiplicidade
1065604,0.1143,-32.340055556,-54.223888889,NDB - MELO,AISGEO,1
1065605,0.285,-21.267052778,-43.764111111,NDB - BARBACENA NDB NOT AVBL BEYOND 60 NM,AISGEO,1
1065606,0.285,-22.197333333,-54.928333333,NDB - DOURA NDB NOT AVBL BEYOND 70 NM,AISGEO,1
1065607,0.375,-19.016219444,-57.664455556,NDB - CORUMBA,AISGEO,1
1065608,0.385,-16.441666667,-39.0825,NDB - PORTO SEGURO NDB NOT AVBL BEYOND 27 NM,AISGEO,1
...,...,...,...,...,...,...
1066219,995.0,-23.631379722,-46.652060278,DME - AEROPORTO DE CONGONHAS 34X,AISGEO,1
1066220,995.0,-5.771808333,-35.350191667,DME - SAO GONCALO 34X,AISGEO,1
1066221,995.0,-9.336946111,-54.952427778,DME - CACHIMBO 34X,AISGEO,1
1066222,995.0,-9.509804722,-35.786944417,DME - IMC 34X,AISGEO,1
